# Redes Neurais Recorrentes

Agora que já sabemos fazer o processamento inicial no texto e construir um classificador, vamos explorar as arquiteturas que permitam processar a sequência de texto mais efetivamente.
[GRU](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU)
[LSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

Como estes algoritmos são capazes de processar sequências de caracteres, podemos considerar tarefas mais desafiadoras.

Considere o problema de **Gerar Texto**.

Na aula passada consideramos a tarefa de classificar o "sentimento" da sentenca. Agora vamos resolver um problema mais desafiador. 

Nesta aula vamos modelar o estilo de "William Shakespeare", treinando um modelo para que complete poemas "como Shakespeare os escreveria".

Basicamente, o problema consiste em prever a próxima palavra dado o restante da frase, por exemplo:

**x** *= "That thereby beauty's rose might never"*, **y** *= "die"*


**x** *= "Or who is he so fond will be the"*, **y** *= "tomb"*

Para tanto, preparamos um dataset com exemplos de sonetos escritos por Shakespeare:


In [1]:
import os

data = open('dataset/text_generation/sonnets.txt').read()
corpus = data.lower().split("\n")
print(corpus[0:5])

['from fairest creatures we desire increase,', "that thereby beauty's rose might never die,", 'but as the riper should by time decease,', 'his tender heir might bear his memory:', 'but thou, contracted to thine own bright eyes,']


Como de costume, precisamos modelar esse problema de maneira que ele seja tratável com uma rede neural.

Agora já sabemos utilizar a classe [Tokenizer](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer) para codificar strings, porém no problema anterior tínhamos que fazer uma classificacão binária de um texto de tamanho máximo constante.

Agora, o modelo deve responder com uma palavra, como podemos modificar nossa rede neural para resolver este problema?

Antes de modificar a rede neural, vamos pensar como transformaremos o texto bruto no dataset para treinamento do modelo.

Uma forma é se aproveitar da funcão [pad_sequence](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) com *padding='pre'* para que normalizemos o tamanho da string de forma que a "última palavra" (isto é, a palavra a ser predita pelo modelo) fique na última posicão e seja fácil de se extrair.

por exemplo:


`
"from fairest creatures we desire increase" -> (Tokenizer) -> 
[1, 2, 3, 4, 5, 6] -> (padding) -> [0, 0, 1, 2, 3, 4, 5, 6]`

Assim, *y* é facilmente acessível no índice `sentenca[-1]`.

Portanto, o primeiro passo é utilizar o tokenizer para que o vocabulário seja definido:


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

#Seu parametro
max_vocab_size = 


tokenizer = Tokenizer(num_words=max_vocab_size, lower=True)
tokenizer.fit_on_texts(corpus)

corpus_seqs = tokenizer.texts_to_sequences(corpus)
corpus_seqs[0:5]


O próximo passo é organizar nossa base de treinamento. 

Como queremos que o modelo seja capaz de prever a próxima palavra em qualquer ponto da frase, vamos considerar todas as subfrases possíveis de se formar com cada linha de soneto.

Por exemplo, para a frase `"from fairest creatures we desire increase"` as seguintes subfrases serão geradas

`
"from fairest"
"from fairest creatures"
"from fairest creatures we"
"from fairest creatures we desire"
"from fairest creatures we desire increase"
`

Ou, em sua versão codificada:

`
[35, 418]
[35, 418, 878]
[35, 418, 878, 167]
[35, 418, 878, 167, 214]
[35, 418, 878, 167, 214, 518]
`

Codifique uma funcao `process_corpus(seqs)` que vai transformar o corpus_seqs definido na célula acima criando as subfrases como descrito.

In [ ]:
def process_corpus(seqs):
    res_seqs = []
    #Seu Codigo Aqui
    return res_seqs

proc_corpus = process_corpus(corpus_seqs)
print(proc_corpus[0:10])

Agora podemos fazer o processamento final das sentencas com [padding](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences).


O faremos de modo que a "palavra a ser predita" fique no último índice, facilitando o processamento posterior quando precisarmos separar a base em "x" e "y".

Note que aqui também devemos determinar o "tamanho máximo da frase", que efetivamente vai determinar até quantas palavras no passado o modelo vai olhar para definir a próxima palavra.

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Seu parametro
max_len = 


padded_dataset = pad_sequences(proc_corpus, maxlen=max_len + 1, padding='pre', truncating='pre')
print(padded_dataset[0:5])

Agora temos os dados organizados de uma maneira mais palatável para uma rede neural.

Diferentemente da rede construída para a tarefa de classificacão de sentimentos, aqui nossa rede neural precisa ter `max_vocab_size` saídas, que efetivamente significa que a saída da rede vai ser uma probabilidade de a próxima palavra ser cada uma das possiblidades em nosso vocábulo.

Para o treinamento, é mais indicado que codifiquemos a palavra a ser prevista com um one-hot encoding, assim a saída da rede vai ser uma probabilidade para cada palavra.

In [ ]:
import tensorflow.keras.utils as ku 
import numpy as np

X = np.array(padded_dataset[:,:-1])
y = padded_dataset[:,-1]
y = ku.to_categorical(y, num_classes = max_vocab_size)


print(X[0:2])
print(y[0:2])

Agora sim, o processamento do texto está terminado e podemos treinar nossa rede neural

In [ ]:
from tensorflow.keras.layers import Bidirectional, Dropout, LSTM, Dense, Embedding
from tensorflow.keras.models import Sequential

embedding_dim = 100
model = Sequential()
#Sua Arquitetura

# Pick an optimizer
optimizer = 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(X, y, epochs=100, verbose=1)

Este modelo pode agora ser utilizado para  prever as próximas palavras:

In [ ]:
def get_next_word(model, frase):
    sequences = tokenizer.texts_to_sequences([frase])
    padded = pad_sequences(sequences, maxlen=max_len , padding='pre', truncating='pre')
    predicted = model.predict_classes(padded)
    
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    return output_word
    
get_next_word(model, 'please, save me in this')
    
    

Inclusive, podemos gerar uma sequencia maior de palavras para formar um novo soneto.

In [ ]:
generate_words = 50

sentence = 'please, save me in this'
for i in range(generate_words):
    sentence = sentence + " " + get_next_word(model, sentence)
print(sentence)

Para uma determinada entrada, o modelo sempre vai responder a mesma palavra, o que não é interessante. Modifique a funcão `get_next_word` para retornar uma palavra com probabilidade igual à especificada pelo modelo.

Teste diversas possibilidades de arquitetura, com modelos bidirecionais, LSTM e GRUs, quais deles obtém um resultado melhor?


Após isso, tente treinar um modelo semelhante usando poemas de [Goncalves Dias](http://www.dominiopublico.gov.br/download/texto/bv000115.pdf)
